# Tensorflow CNN with Class,Ensemble

In [1]:
import tensorflow as tf

In [2]:
import random
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from copy import copy
import os
tf.set_random_seed(777)

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
epochs=20
batch_size=100
learning_rate=0.001

In [5]:
class Model:
    def __init__(self, sess, name):
        self.sess=sess
        self.name=name
        self._build_net()
        
    def _build_net(self):
        self.training= tf.placeholder(tf.bool)
        self.X = tf.placeholder(tf.float32,[None, 784])
        X_img = tf.reshape(self.X, [-1,28,28,1])
        self.Y = tf.placeholder(tf.float32,[None,10])
        
        conv1 = tf.layers.conv2d(inputs = X_img, filters = 32, kernel_size=[3,3], padding = "SAME", activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size = [2, 2], padding="SAME", strides=2)
        drop1 = tf.layers.dropout(inputs=pool1, rate=0.7, training = self.training)
        
        conv2 = tf.layers.conv2d(inputs = drop1, filters = 64, kernel_size=[3, 3], padding = "SAME", activation=tf.nn.relu)
        pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size = [2, 2], padding="SAME", strides=2)
        drop2 = tf.layers.dropout(inputs=pool2, rate=0.7, training = self.training)
        
        conv3 = tf.layers.conv2d(inputs = drop2, filters = 128, kernel_size=[3, 3], padding = "SAME", activation=tf.nn.relu)
        pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size = [2, 2], padding="SAME", strides=2)
        drop3 = tf.layers.dropout(inputs=pool3, rate=0.7, training = self.training)
        
        flat = tf.reshape(drop3, [-1,128*4*4])
        dense4 = tf.layers.dense(inputs=flat,units=625,activation=tf.nn.relu,kernel_initializer = tf.contrib.layers.variance_scaling_initializer())
        drop4 = tf.layers.dropout(inputs=dense4,rate=0.6,training = self.training)
        batch4 = tf.layers.batch_normalization(drop4,training= True)

        dense5 = tf.layers.dense(inputs=batch4,units=200,activation=tf.nn.relu,kernel_initializer = tf.contrib.layers.variance_scaling_initializer())
        drop5 = tf.layers.dropout(inputs=dense5,rate=0.6,training = self.training)
        batch5 = tf.layers.batch_normalization(drop5,training= True)
        
        self.logits = tf.layers.dense(inputs=batch5,units=10)
        
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits,labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
        
        correct_prediction = tf.equal(tf.argmax(self.logits,1),tf.argmax(self.Y,1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        
        
    def predict(self,x_test,training=False):
        return self.sess.run(self.logits,feed_dict={self.X:x_test, self.training:training})
        
    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy,feed_dict={self.X:x_test, self.Y:y_test, self.training:training})
        
    def train(self,x_test,y_test,training=True):
        return self.sess.run([self.cost,self.optimizer],feed_dict={self.X:x_test,self.Y:y_test, self.training:training})

In [6]:
sess = tf.Session()

models=[]
num_models=7
for m in range(num_models):
    models.append(Model(sess,"model"+str(m)))

sess.run(tf.global_variables_initializer())


In [7]:
print("Learning start!!")

for epoch in range(epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        
        for m_idx,m in enumerate(models):
            c,_ = m.train(batch_xs,batch_ys)
            avg_cost_list[m_idx] += c/total_batch
    print("epoch : ", "%04d"%(epoch+1), " cost : ", avg_cost_list)
    
print("Learning finished!!")


Learning start!!
epoch :  0001  cost :  [ 1.48335613  1.46950979  1.44522729  1.45866581  1.43859874  1.49825155
  1.41484424]
epoch :  0002  cost :  [ 0.61674833  0.59948313  0.61801164  0.6070387   0.60974949  0.61686311
  0.6000339 ]
epoch :  0003  cost :  [ 0.44496018  0.42917468  0.44364652  0.47278734  0.48406952  0.44436493
  0.44767544]
epoch :  0004  cost :  [ 0.36205577  0.350574    0.35321389  0.38492298  0.40116563  0.36066929
  0.37510111]
epoch :  0005  cost :  [ 0.3126456   0.30432752  0.30606643  0.32713716  0.34002169  0.30535766
  0.32183612]
epoch :  0006  cost :  [ 0.28687012  0.27487363  0.2782214   0.29522714  0.30476598  0.2781608
  0.28944802]
epoch :  0007  cost :  [ 0.26373795  0.25783114  0.26067793  0.2721159   0.27729029  0.25647446
  0.2720459 ]
epoch :  0008  cost :  [ 0.24660888  0.24129316  0.23959809  0.25653652  0.25648285  0.23670561
  0.24990426]
epoch :  0009  cost :  [ 0.22661195  0.22285208  0.22734903  0.237676    0.24159321  0.22382934
  0.2356

In [10]:
sample_data = mnist.test.images[5000:10000]
sample_label = mnist.test.labels[5000:10000]


test_size = len(sample_label)
prediction = np.zeros(test_size*10).reshape(test_size,10)

for m_idx,m in enumerate(models):
    print(m_idx, "Accuracy = ", m.get_accuracy(sample_data,sample_label))
    p = m.predict(sample_data)
    prediction +=p

ensemble_correct = tf.equal(tf.argmax(prediction,1),tf.argmax(sample_label,1))
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct,tf.float32))

print("Ensemble Accuracy = ", sess.run(ensemble_accuracy) )

0 Accuracy =  0.99
1 Accuracy =  0.9934
2 Accuracy =  0.9912
3 Accuracy =  0.991
4 Accuracy =  0.9896
5 Accuracy =  0.991
6 Accuracy =  0.9918
Ensemble Accuracy =  0.9914
